In [3]:
import os
import csv
import numpy as np

In [8]:
main_dir ="/Users/08volt/Desktop/StremingML/TEST/Results_union"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [9]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [11]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0 = []
                class1 = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        
                        try:
                            daily0.append(float(data_row[9]))
                        except:
                            if data_row[9] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[10]))
                        except:
                            if data_row[10] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0.append(np.mean(daily0))
                        class1.append(np.mean(daily1))
                            
                        
                    
                assert len(class0) > 0
                assert len(class1) > 0
                assert len(class1) == len(class0)
                if np.mean(class0) != np.mean(class0):
                    print(class0)
                if np.mean(class1) != np.mean(class1):
                    print(class1)
                if alg == "OSHAT":
                    alg = "CSMOTE"
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0"] ={} 
                final_dict[old][unbalance][dtype][alg]["class1"] = {}
                final_dict[old][unbalance][dtype][alg]["class0"]["mean"] = np.mean(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["mean"] = np.mean(class1)
                final_dict[old][unbalance][dtype][alg]["class0"]["std"] =np.std(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["std"] = np.std(class1)

                print(np.mean(class0))
                print(np.mean(class1))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
99.04130893811295
98.10819729063016
sea_P(y) 1 sudden WEOB1
98.97942226181465
97.83921843449522
sea_P(y) 1 sudden OOB
98.98612670033285
98.03087014819405
sea_P(y) 1 sudden ESOS_ELM
89.30415619559611
73.42351874845741
sea_P(y) 1 sudden improved_UOB
98.81573147725297
96.9240271514656
sea_P(y) 1 sudden WEOB2
98.6244127997512
96.19870135316566
sea_P(y) 1 sudden UOB
98.98612670033285
98.03087014819405
sea_P(y) 1 sudden OSHAT
98.74740154470427
97.13040882292873
sea_P(y) 1 sudden RebalanceStream
98.4522316004296
93.46004270685152
sea_P(y) 3 incremental improved_OOB
88.08041188023472
80.58624789587222
sea_P(y) 3 incremental WEOB1
88.0473140329549
80.63956462559813
sea_P(y) 3 incremental OOB
87.78576702893473
80.36644102090415
sea_P(y) 3 incremental ESOS_ELM
82.41503576802805
63.45563049312749
sea_P(y) 3 incremental improved_UOB
87.73204260284595
80.2582474829982
sea_P(y) 3 incremental WEOB2
87.13733641570286
79.81299251202663
sea_P(y) 3 incremental UOB
87.6816893

88.9539266307108
66.31599889495646
sea_P(y_x) 2 incremental OOB
89.0813931637065
67.50698321213761
sea_P(y_x) 2 incremental ESOS_ELM
87.54162643112556
51.675839884918844
sea_P(y_x) 2 incremental improved_UOB
87.3328771880704
62.56117230599576
sea_P(y_x) 2 incremental WEOB2
87.30157996253263
60.28309488580048
sea_P(y_x) 2 incremental UOB
85.12746657850657
60.12601746519365
sea_P(y_x) 2 incremental OSHAT
87.46231358011173
60.99464766018574
sea_P(y_x) 2 incremental RebalanceStream
90.91032302421948
51.31017005743174
sea_P(y_x) 4 sudden improved_OOB
84.98443973509949
78.38035652617066
sea_P(y_x) 4 sudden WEOB1
84.88722679721056
78.22086326320559
sea_P(y_x) 4 sudden OOB
84.77730113892997
79.2364676934806
sea_P(y_x) 4 sudden ESOS_ELM
81.21121484849324
61.83718447652819
sea_P(y_x) 4 sudden improved_UOB
84.419685975693
77.59199342741667
sea_P(y_x) 4 sudden WEOB2
82.4376348103982
69.42351001442499
sea_P(y_x) 4 sudden UOB
83.32791008804074
77.79005542537746
sea_P(y_x) 4 sudden OSHAT
84.366290035

98.88529064099632
96.941864075638
sine_P(y) 1 sudden ESOS_ELM
94.16070990992786
76.85256496051791
sine_P(y) 1 sudden improved_UOB
97.1935718576103
92.20395845486993
sine_P(y) 1 sudden WEOB2
96.9514535412375
92.12930276902259
sine_P(y) 1 sudden UOB
98.01880455433135
94.15706560410067
sine_P(y) 1 sudden OSHAT
95.1037404216498
90.95565798916955
sine_P(y) 1 sudden RebalanceStream
94.87462344631152
94.11945066271572
sine_P(y) 2 incremental improved_OOB
99.29619565616777
98.42540364220574
sine_P(y) 2 incremental WEOB1
99.19876266981477
98.09646693658878
sine_P(y) 2 incremental OOB
99.33197016305141
98.44344387366252
sine_P(y) 2 incremental ESOS_ELM
95.1875442715925
79.32635522738295
sine_P(y) 2 incremental improved_UOB
98.56097673928278
96.84373273135517
sine_P(y) 2 incremental WEOB2
98.3511321138681
96.38275111904463
sine_P(y) 2 incremental UOB
98.55683959548938
96.69352921941831
sine_P(y) 2 incremental OSHAT
97.637939598043
96.67757574789115
sine_P(y) 2 incremental RebalanceStream
97.06340

99.38471967763948
98.58312840252233
sine_P(x_y) 3 incremental OOB
99.33112018525699
98.4712411421343
sine_P(x_y) 3 incremental ESOS_ELM
94.26518932554933
78.19749069091901
sine_P(x_y) 3 incremental improved_UOB
99.015704052556
97.76908076397868
sine_P(x_y) 3 incremental WEOB2
98.89629829815445
97.49579167598293
sine_P(x_y) 3 incremental UOB
98.55314905435208
96.79616746617275
sine_P(x_y) 3 incremental OSHAT
99.04782134972837
97.81000762879361
sine_P(x_y) 3 incremental RebalanceStream
98.56795698798973
96.63728717996204
sine_P(x_y) 4 sudden improved_OOB
99.31407371998966
98.98293442056965
sine_P(x_y) 4 sudden WEOB1
99.2322414975753
98.86737831768696
sine_P(x_y) 4 sudden OOB
99.35443629525426
99.03234628120268
sine_P(x_y) 4 sudden ESOS_ELM
92.085486527406
78.93582104822516
sine_P(x_y) 4 sudden improved_UOB
98.96277152634195
98.48284670037606
sine_P(x_y) 4 sudden WEOB2
98.91653346501666
98.41166868844925
sine_P(x_y) 4 sudden UOB
99.35443629525426
99.03234628120268
sine_P(x_y) 4 sudden OSH

In [7]:
old = final_dict.copy()
old

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': 99.04130893811295,
     'class1': 98.10819729063016},
    'WEOB1': {'class0': 98.97942226181465, 'class1': 97.83921843449522},
    'OOB': {'class0': 98.98612670033285, 'class1': 98.03087014819405},
    'ESOS_ELM': {'class0': 89.30415619559611, 'class1': 73.42351874845741},
    'improved_UOB': {'class0': 98.81573147725297, 'class1': 96.9240271514656},
    'WEOB2': {'class0': 98.6244127997512, 'class1': 96.19870135316566},
    'UOB': {'class0': 98.98612670033285, 'class1': 98.03087014819405},
    'CSMOTE': {'class0': 98.74740154470427, 'class1': 97.13040882292873},
    'RebalanceStream': {'class0': 98.4522316004296,
     'class1': 93.46004270685152}},
   'incremental': {'improved_OOB': {'class0': 99.07781073529245,
     'class1': 98.36413751297053},
    'WEOB1': {'class0': 99.00395733265307, 'class1': 98.04731708900161},
    'OOB': {'class0': 99.0202469306703, 'class1': 98.31815573804992},
    'ESOS_ELM': {'class0': 89.7427966771706

# NEW

In [8]:
final_dict = old.copy()

In [12]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0"] = []
            alg_dtypes["sudden"]["class0"] = []
            alg_dtypes["periodic"]["class0"] = []
            alg_dtypes["incremental"]["class1"] = []
            alg_dtypes["sudden"]["class1"] = []
            alg_dtypes["periodic"]["class1"] = []
            
            
            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue

                    try:
                        daily0.append(float(data_row[9]))
                    except:
                        if data_row[9] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[10]))
                    except:
                        if data_row[10] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        if alg == "ESOS_ELM" and new == "appearing-minority":
                            print(daily0)
                            print(daily1)
                        alg_dtypes[result[:result.index("_")]]["class0"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                if len(i["class0"]) < 10:
                    print(f"N EXP {len(i['class0'])} in 0")
                if len(i["class1"]) < 10:
                    print(f"N EXP {len(i['class1'])} in 1")
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0"] ={} 
                final_dict[new][unbalance][k][alg]["class1"] = {}
                final_dict[new][unbalance][k][alg]["class0"]["mean"] = np.mean(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["mean"] = np.mean(i["class1"])
                final_dict[new][unbalance][k][alg]["class0"]["std"] =np.std(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["std"] = np.std(i["class1"])

                print(k)
                print(np.mean(i["class1"]))
                print(np.mean(i["class0"]))

jitter 1 improved_OOB
incremental
57.37133226312047
92.56181751723093
sudden
75.55851762348355
95.70163543712847
periodic
57.37133226312047
92.56181751723093
jitter 1 WEOB1
incremental
53.6243098407444
90.4801850117582
sudden
72.8682131895415
94.70571906938414
periodic
53.6243098407444
90.4801850117582
jitter 1 CSMOTE
incremental
42.77060637417368
86.0907328276071
sudden
47.74465185542761
88.01308046367015
periodic
42.77060637417368
86.0907328276071
jitter 1 OOB
incremental
54.83374234342974
91.18250060393169
sudden
73.58168785955442
94.725496172701
periodic
54.83374234342974
91.18250060393169
jitter 1 ESOS_ELM
incremental
34.1052838404195
84.46844793787247
sudden
55.50938218131704
93.19340712062409
periodic
33.59323904616951
84.2229182286193
jitter 1 improved_UOB
incremental
44.149803923449596
85.00700687090594
sudden
55.52449705489572
88.76459014182113
periodic
44.149803923449596
85.00700687090594
jitter 1 WEOB2
incremental
43.1823170557803
84.25716985850703
sudden
47.218436957326006

incremental
41.61808282242059
92.2959620938453
sudden
38.68829011005202
92.6661720045237
periodic
78.39361160656044
88.47779973321963
disappearing-minority 3 CSMOTE
incremental
41.05504253089475
92.53990786068297
sudden
38.63563806683284
92.81597442277179
periodic
76.68147001187914
87.67874797845482
disappearing-minority 3 OOB
incremental
41.921994450520046
92.65073033911958
sudden
38.95956029436961
92.7679407825923
periodic
77.88794424634645
88.01724783329746
disappearing-minority 3 ESOS_ELM
incremental
25.60951158194164
89.72446376154852
sudden
23.094213956494094
90.60992044781736
periodic
57.1376436620748
82.39269151501252
disappearing-minority 3 improved_UOB
incremental
40.34659079190571
85.4732415699423
sudden
37.58719003347221
84.32806089993059
periodic
76.95522570787871
87.20036878335608
disappearing-minority 3 WEOB2
incremental
39.72899594306513
85.43164470873606
sudden
36.92199002157004
83.54914045303512
periodic
76.04036069820827
86.51438912313539
disappearing-minority 3 UOB


incremental
38.602102011746695
94.80259451832714
sudden
35.0021959731456
94.32978767506424
periodic
6.616451898083419
99.05445865245784
appearing-minority 2 CSMOTE
incremental
36.58915977598559
94.19050831335787
sudden
32.63663900704978
93.31806573333246
periodic
5.937710176418852
99.39906609072537
appearing-minority 2 OOB
incremental
40.17386376906564
95.50286717529346
sudden
35.334113861465326
94.49408518387642
periodic
7.231709757073562
99.55479732385457
appearing-minority 2 ESOS_ELM
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 97.2356386805056, 94.7423819798958, 89.2702912191341, 89.31931377974544, 89.196675900277, 89.12603597860132, 89.09098975200746, 88.98345853532176, 88.95762940401973, 88.93516590309594, 88.9793501591296]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 89.11066755378133, 88.77766655544434, 88.7941579864329, 88.9536659171

incremental
42.3558178485214
94.25823500435949
sudden
39.8621697866793
94.09339175639357
periodic
6.805785779148709
98.56653290230045
appearing-minority 3 CSMOTE
incremental
41.483971972258075
94.08573897558892
sudden
39.52286839414388
94.07466192786512
periodic
6.380336459744504
98.76028982993611
appearing-minority 3 OOB
incremental
44.505736503884584
95.11878574843885
sudden
39.677964886912676
93.84276181303744
periodic
7.457271234784618
99.38923448595946
appearing-minority 3 ESOS_ELM
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 96.26556016597512, 92.12513484358144, 82.7117053718039, 82.33909871749617, 82.22361809045226, 82.29408822058089, 82.22269333144891, 82.35294117647058, 82.39840061824842, 82.38234559007306, 82.4067420134579]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 82.31140402494998, 81.88046302858493, 82.11717552752563, 82.12759

incremental
46.76995578451033
94.42242044263989
sudden
42.633579062978086
93.6231275031931
periodic
7.250671267153879
98.70008064628692
appearing-minority 4 CSMOTE
incremental
44.83969503291134
92.99022128784368
sudden
41.958619467806486
93.33139677028927
periodic
6.895902654652322
98.95603321035337
appearing-minority 4 OOB
incremental
47.70387369498626
94.87092829984559
sudden
42.63580501733282
93.59073238144374
periodic
7.535038277185071
98.99539329914947
appearing-minority 4 ESOS_ELM
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 94.7700726086499, 88.96291361314678, 75.17163899637997, 75.41269544633401, 75.19241169863128, 75.03124218945264, 74.86548039142077, 74.82527694907475, 74.73829250398153, 74.82592980864014, 74.92008783389387]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 74.27709328639678, 74.15051598288447, 74.60290921250626, 74.8670

incremental
64.60346190666675
93.80237423128128
sudden
64.61170712052261
93.77527150966282
periodic
74.24920625580289
96.09922740985385
appearing-clusters 1 CSMOTE
incremental
65.70502492757362
94.98153743256016
sudden
66.6381136936992
95.23742120310371
periodic
75.21938346241538
96.53459870548633
appearing-clusters 1 OOB
incremental
66.7160010795512
94.46259420256693
sudden
65.2259235861293
93.89588789921017
periodic
75.76469090011823
96.4392046771861
appearing-clusters 1 ESOS_ELM
incremental
36.69139477848908
86.69801492755002
sudden
36.56662908882637
86.7317946723961
periodic
40.58515853354128
88.5761424765391
appearing-clusters 1 improved_UOB
incremental
60.68772451675736
92.66321215230877
sudden
60.400750012063575
92.54644979877482
periodic
70.03612012547833
95.09943891478325
appearing-clusters 1 WEOB2
incremental
59.245775636311954
92.59789790168233
sudden
58.25603694252486
92.46330786885241
periodic
68.10871444803504
94.63467834363033
appearing-clusters 1 UOB
incremental
60.6933

incremental
78.79612707090124
87.70001182332994
sudden
78.83839354397074
87.77015824859163
periodic
78.55455450917239
87.41906691625817
shapeshift 3 CSMOTE
incremental
76.91484266792258
86.55243194323714
sudden
76.950992055699
86.64742716648662
periodic
76.37739975516206
85.74491007796937
shapeshift 3 OOB
incremental
79.18919848382328
87.64891040029165
sudden
79.14935451048773
87.60009918294931
periodic
78.52421441370934
86.98317394969072
shapeshift 3 ESOS_ELM
incremental
59.75183092335478
82.25603240805012
sudden
59.480195816889974
82.22406749668687
periodic
57.79331110258553
81.17344346029468
shapeshift 3 improved_UOB
incremental
77.31269052495864
86.3822220629119
sudden
77.26812604627743
86.37278322737079
periodic
76.77616996274682
85.85217757212624
shapeshift 3 WEOB2
incremental
76.58767959786789
85.73840887705805
sudden
77.03372835147243
86.05632167440015
periodic
76.5276233708689
85.75809973547362
shapeshift 3 UOB
incremental
76.11230499888345
84.69404325984922
sudden
75.90178157

incremental
38.025794831006436
87.19361531712447
sudden
36.412349465584484
86.78396677705571
periodic
41.56955273587115
88.98103399188516
splitting-clusters 1 improved_UOB
incremental
60.13257206714892
92.3302549242721
sudden
57.703316608596516
91.86548095226121
periodic
66.20434436256325
94.27992347869937
splitting-clusters 1 WEOB2
incremental
60.08052737335921
92.75170998661869
sudden
57.90394337660634
92.35341773716586
periodic
66.07250229111264
94.2352488139467
splitting-clusters 1 UOB
incremental
58.32440127596463
91.44779304385199
sudden
54.10311405328175
89.87615312805904
periodic
65.35611355274855
94.03619209217189
splitting-clusters 1 RebalanceStream
incremental
62.08296171292413
96.72315044050393
sudden
64.76648890541108
96.76123170272922
periodic
70.83029320688438
97.15396276870828
splitting-clusters 2 improved_OOB
incremental
78.64666898961379
92.97586871142627
sudden
78.44489443592454
92.90514198463225
periodic
83.23338712643371
94.97943390692105
splitting-clusters 2 WEOB1

incremental
72.87879470720131
81.36063060838251
sudden
68.61892587053102
75.77937234326296
periodic
72.87516609874498
82.53072314463382
clusters-movement 3 RebalanceStream
incremental
72.92218034277849
88.02635302067209
sudden
60.06016112154244
86.32488422233205
periodic
71.16668525620797
86.78785323781607
clusters-movement 4 improved_OOB
incremental
84.3439818795573
87.28869799806486
sudden
84.37136837488492
87.18868375056661
periodic
83.57212747656612
86.3974977158109
clusters-movement 4 WEOB1
incremental
84.07591728497957
86.86603789563932
sudden
84.02323281651061
86.78300052289191
periodic
83.15388630596883
85.9553873643641
clusters-movement 4 CSMOTE
incremental
76.50554207482254
79.2437461198857
sudden
74.42917059383696
78.63028398734562
periodic
82.2720538937077
85.38299704527745
clusters-movement 4 OOB
incremental
83.82953016694071
86.87845853672658
sudden
84.04981180798018
86.99393726366894
periodic
83.08697707741443
85.89978463845148
clusters-movement 4 ESOS_ELM
incremental
64

In [13]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': {'mean': 99.04130893811295,
      'std': 0.0},
     'class1': {'mean': 98.10819729063016, 'std': 1.4210854715202004e-14}},
    'WEOB1': {'class0': {'mean': 98.97942226181465, 'std': 0.0},
     'class1': {'mean': 97.83921843449522, 'std': 0.0}},
    'OOB': {'class0': {'mean': 98.98612670033285, 'std': 0.0},
     'class1': {'mean': 98.03087014819405, 'std': 0.0}},
    'ESOS_ELM': {'class0': {'mean': 89.30415619559611,
      'std': 0.10278132708036258},
     'class1': {'mean': 73.42351874845741, 'std': 0.5217861393667154}},
    'improved_UOB': {'class0': {'mean': 98.81573147725297, 'std': 0.0},
     'class1': {'mean': 96.9240271514656, 'std': 0.0}},
    'WEOB2': {'class0': {'mean': 98.6244127997512, 'std': 0.0},
     'class1': {'mean': 96.19870135316566, 'std': 1.4210854715202004e-14}},
    'UOB': {'class0': {'mean': 98.98612670033285, 'std': 0.0},
     'class1': {'mean': 98.03087014819405, 'std': 0.0}},
    'CSMOTE': {'class0': {'me

In [4]:
# def stats_of(_dict ):
#     values_0 = []
#     values_1 = []
#     for alg, values in _dict.items():
#         if "ELM" in alg:
#             continue
#         values_0.append(values["class0"])
#         values_1.append(values["class1"])
#     return np.mean(values_0),np.mean(values_1), np.max(values_0), np.max(values_1), np.min(values_0), np.min(values_1)


In [14]:

with open('F1_withSTD.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","F1_0", "F1_1","F1_0_std", "F1_1_std"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                #mean0, mean1,max0,max1,min0,min1 = stats_of(algs)
                for alg, values in algs.items():
                    
                    print([drift,unbalance,dtype,alg])
                    print(values)

                    writer.writerow([drift,unbalance,dtype,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0': {'mean': 99.04130893811295, 'std': 0.0}, 'class1': {'mean': 98.10819729063016, 'std': 1.4210854715202004e-14}}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0': {'mean': 98.97942226181465, 'std': 0.0}, 'class1': {'mean': 97.83921843449522, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0': {'mean': 98.98612670033285, 'std': 0.0}, 'class1': {'mean': 98.03087014819405, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0': {'mean': 89.30415619559611, 'std': 0.10278132708036258}, 'class1': {'mean': 73.42351874845741, 'std': 0.5217861393667154}}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0': {'mean': 98.81573147725297, 'std': 0.0}, 'class1': {'mean': 96.9240271514656, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0': {'mean': 98.6244127997512, 'std': 0.0}, 'class1': {'mean': 96.19870135316566, 'std': 1.4210854715202004e-14}}
['sea_P(y)', 1, 'sudden', 'UOB']
{'class0': {'mean': 98.98612670033285, 'std': 0.0}, 'clas

['appearing-minority', 1, 'incremental', 'improved_OOB']
{'class0': {'mean': 96.50349278342587, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 32.41267805275402, 'std': 0.0}}
['appearing-minority', 1, 'incremental', 'WEOB1']
{'class0': {'mean': 95.35612213259678, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 29.562879919907896, 'std': 3.552713678800501e-15}}
['appearing-minority', 1, 'incremental', 'CSMOTE']
{'class0': {'mean': 94.14075375898969, 'std': 0.0}, 'class1': {'mean': 25.213870743268643, 'std': 0.0}}
['appearing-minority', 1, 'incremental', 'OOB']
{'class0': {'mean': 95.85015565415891, 'std': 0.0}, 'class1': {'mean': 31.42376616617948, 'std': 3.552713678800501e-15}}
['appearing-minority', 1, 'incremental', 'ESOS_ELM']
{'class0': {'mean': 97.44356423338117, 'std': 0.0}, 'class1': {'mean': 0.0, 'std': 0.0}}
['appearing-minority', 1, 'incremental', 'improved_UOB']
{'class0': {'mean': 91.02141293232448, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 20.87091659569015

['splitting-clusters', 3, 'sudden', 'improved_UOB']
{'class0': {'mean': 91.20546386984952, 'std': 0.0}, 'class1': {'mean': 83.8488394011927, 'std': 1.4210854715202004e-14}}
['splitting-clusters', 3, 'sudden', 'WEOB2']
{'class0': {'mean': 91.31802280970919, 'std': 0.0}, 'class1': {'mean': 83.79830707748121, 'std': 0.0}}
['splitting-clusters', 3, 'sudden', 'UOB']
{'class0': {'mean': 91.08777545408712, 'std': 0.0}, 'class1': {'mean': 83.45513898225018, 'std': 1.4210854715202004e-14}}
['splitting-clusters', 3, 'sudden', 'RebalanceStream']
{'class0': {'mean': 92.51681716091468, 'std': 0.0}, 'class1': {'mean': 83.9917249042236, 'std': 0.0}}
['splitting-clusters', 3, 'periodic', 'improved_OOB']
{'class0': {'mean': 94.55265662552742, 'std': 0.0}, 'class1': {'mean': 88.86001626337834, 'std': 1.4210854715202004e-14}}
['splitting-clusters', 3, 'periodic', 'WEOB1']
{'class0': {'mean': 94.33095918578479, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 88.54087067179694, 'std': 0.0}}
['splitting-

# REALS

In [15]:
real_dict = {}
real_dict["KDDCup"] = {}
real_dict["PAKDD"] = {}
real_dict["Elec"] = {}

for i in real_dict.keys():
    path = f"/Users/08volt/Desktop/StremingML/TEST/Results_union/Real/{i}"
    for alg in os.listdir(path):
        class0 = []
        if "DS_Store" in alg or "ipynb" in alg:
            continue
        class1 = []
        if alg not in real_dict[i]:
            real_dict[i][alg] = {}
        
            
        for exp in os.listdir(path + "/" + alg): 
            if "err" not in exp or "DS_Store" in exp:
                continue
                
            data = import_csv(path + "/" + alg + "/"+ exp)
            daily0 = []
            daily1 = []
            start = False

            for data_row in data:
                if data_row[1] == "learning evaluation instances":
                    start = True
                    continue
                if not start:
                    continue

                try:
                    daily0.append(float(data_row[9]))
                except:
                    if data_row[9] == "?":
                        daily0.append(0.0)
                    else:
                        print("ZERO")
                        print(data_row)
                try:
                    daily1.append(float(data_row[10]))
                except:
                    if data_row[10] == "?":
                        daily1.append(0.0)
                    else:
                        print("ONE")
                        print(data_row)
            try:
                if len(daily0)>0 and len(daily1)>0:
                    class0.append(np.mean(daily0))
                    class1.append(np.mean(daily1))
            except:
                print(f"error with {result}")
        real_dict[i][alg]["class0"] = {}
        real_dict[i][alg]["class1"] = {}
        real_dict[i][alg]["class0"]["mean"] = np.mean(class0)
        real_dict[i][alg]["class1"]["mean"] = np.mean(class1)
        real_dict[i][alg]["class0"]["std"] = np.std(class0)
        real_dict[i][alg]["class1"]["std"] = np.std(class1)
        
                
                

In [16]:
real_dict

{'KDDCup': {'improved_OOB': {'class0': {'mean': 97.92131092513542,
    'std': 1.4210854715202004e-14},
   'class1': {'mean': 91.26893835964778, 'std': 0.0}},
  'WEOB1': {'class0': {'mean': 95.63253800339828, 'std': 0.0},
   'class1': {'mean': 91.14788385158384, 'std': 0.0}},
  'OOB': {'class0': {'mean': 97.71239503010152, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 92.83696092715991, 'std': 0.0}},
  'ESOS_ELM': {'class0': {'mean': 83.12720197527338, 'std': 0.0},
   'class1': {'mean': 0.0, 'std': 0.0}},
  'improved_UOB': {'class0': {'mean': 92.06415723668195,
    'std': 1.4210854715202004e-14},
   'class1': {'mean': 89.58295496510378, 'std': 0.0}},
  'WEOB2': {'class0': {'mean': 92.4518633128902, 'std': 0.0},
   'class1': {'mean': 81.49641525620021, 'std': 0.0}},
  'UOB': {'class0': {'mean': 95.05175395125022, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 94.35237962976271, 'std': 0.0}},
  'OSHAT': {'class0': {'mean': 97.32720428082052, 'std': 0.0},
   'class1': {'mean

In [17]:
with open('F1XF1_real_withSTD.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["ds","alg","F1_0", "F1_1","F1_0_std", "F1_1_std"])
    for ds, dicts in real_dict.items():
        
        for alg, values in dicts.items():
            
            if alg == "OSHAT":
                alg = "CSMOTE"
            writer.writerow([ds,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])
